In [55]:
import pandas as pd

In [56]:
df = pd.read_excel('/content/jogja_lagi.xlsx')

In [57]:
df.head()

,Keywords,Address,Price,Info,Info1,Info2,Info3,Info4,Info5,Title,...,Field4,Field6,Text1,Text2,Text3,lokasi 1,lokasi 2,lokasi 3,lokasi 4,lokasi 5
0,Premium\n \n ...,Yogyakarta,Rp825.000.000,2,Kamar tidur,40 m²,Bangunan,66 m²,Lahan,Yogyakarta,...,66 m²,rincian,rincian\n \n\n \n \n \n ...,Fasilitas\n \n ...,NaN,malioboro,Stasiun Yogyakarta,Universitas Ahmad Dahlan,Stasiun Tugu,Masjid Gedhe Kauman
1,Premium\n \n ...,"Jetis, Yogyakarta",Rp1.680.000.000,4,Kamar tidur,140 m²,Bangunan,500 m²,Lahan,"Jetis, Yogyakarta",...,500 m²,rincian,rincian\n \n\n \n \n \n ...,Fasilitas\n \n ...,Di sekitar lokasi\n \n\n ...,malioboro,Malioboro Mall,Stasiun Yogyakarta,Galeria Mall,Stasiun Tugu
2,Premium\n \n ...,Yogyakarta,Rp930.000.000,2,Kamar tidur,66 m²,Bangunan,92 m²,Lahan,"Jalan Gedongan Kuning Selatan, Purbayan Kel., ...",...,92 m²,rincian,rincian\n \n\n \n \n \n ...,Fasilitas\n \n ...,Di sekitar lokasi\n \n\n ...,malioboro,Stasiun Yogyakarta,Universitas Ahmad Dahlan,Stasiun Tugu,Masjid Gedhe Kauman
3,Premium\n \n ...,"Mergangsan, Yogyakarta",Rp4.500.000.000,4,Kamar tidur,350 m²,Bangunan,230 m²,Lahan,jalan parangtritis jogja\n ...,...,230 m²,rincian,rincian\n \n\n \n \n \n ...,Fasilitas\n \n ...,Di sekitar lokasi\n \n\n ...,Masjid Jogokariyan,NaN,NaN,NaN,NaN
4,New\n\n \n \...,"Kraton, Yogyakarta",Rp1.300.000.000,3,Kamar tidur,90 m²,Bangunan,100 m²,Lahan,"Kraton, Yogyakarta",...,100 m²,rincian,rincian\n \n\n \n \n \n ...,NaN,Di sekitar lokasi\n \n\n ...,malioboro,Masjid Jogokariyan,Masjid Gedhe Kauman,SMAN 1 Teladan Yogyakarta,Gereja Saint Francis Xaverius Yogyakarta


In [58]:
df.duplicated().sum()

35

In [59]:
df = df.drop_duplicates()

In [60]:
df.duplicated().sum()

0

In [61]:
df['Price'] = df['Price'].str.replace('Rp', '').str.replace('.', '').astype(int)

<ipython-input-61-661097e950d7>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Price'] = df['Price'].str.replace('Rp', '').str.replace('.', '').astype(int)


In [62]:
df = df.rename(columns={"Price": "harga"})
df = df.rename(columns={"Info": "kamar_tidur"})
df = df.rename(columns={"Info2": "bangunan",'Info4':'lahan'})

In [63]:
df['lahan'] = df['lahan'].str.replace('m²', '', regex=False).str.strip()
df['bangunan'] = df['bangunan'].str.replace('m²', '', regex=False).str.strip()

In [64]:
df = df.drop(['Info1','Info3','Info5'],axis=1)

In [65]:
df.replace('\n', '', regex=True, inplace=True)

In [66]:
df.replace('\n', '', regex=True, inplace=True)

In [67]:
df.replace('\n', '', regex=True, inplace=True)

In [68]:
df = df.drop(['Field1','Field2','Field3','Field4','Field6'],axis=1)

In [69]:
df1=df.copy()

In [70]:
import re

In [71]:
def extract_info(text, key):
    pattern = re.compile(f'{key}[\s:]+([\d/]+)')
    match = pattern.search(text)
    return match.group(1) if match else None
keys_to_extract = ['Lahan Parkir', 'Jumlah Lantai','Kamar mandi']
extracted_info = {key: [] for key in keys_to_extract}
for index, row in df1.iterrows():
    for key in keys_to_extract:
        extracted_info[key].append(extract_info(row['Text1'], key))
for key in keys_to_extract:
    df1[key] = extracted_info[key]

In [72]:
df2 = df1.copy()

In [73]:
# Fasilitas yang ingin diekstrak
fasilitas = ['Sistem Alarm', 'Gym', 'Internet broadband/Wifi', 'TV kabel', 'Pemanas ruangan',
             'Pendingin ruangan (AC)', 'Air Panas', 'Telepon', 'Televisi', 'Kitchen set',
             'Garasi', 'Secure parking', 'Kolam renang', 'Lapangan tenis', 'Balkon',
             'Dek', 'Halaman terbuka', 'Area hiburan outdoor', 'Pagar penuh', 'Taman',
             'Keamanan 24 jam', 'Taman bermain anak']

# Fungsi untuk mengekstrak fasilitas
def extract_facilities(text, facilities):
    if pd.isna(text):  # Handle nilai kosong (NaN)
        return ['N'] * len(facilities)

    extracted_facilities = []
    for facility in facilities:
        if facility in text:
            extracted_facilities.append('Y')
        else:
            extracted_facilities.append('N')
    return extracted_facilities

# Menerapkan fungsi pada DataFrame
df2[fasilitas] = pd.DataFrame(df2['Text2'].apply(lambda x: extract_facilities(x, fasilitas)).tolist(), index=df.index)

In [74]:
df3=df2.copy()

In [75]:
# Menggabungkan semua nilai unik dari setiap kolom yang di-for
unique_values = pd.unique(df3[['lokasi 1', 'lokasi 2', 'lokasi 3', 'lokasi 4', 'lokasi 5']].values.ravel('K'))

# Membuat DataFrame hasil dengan indeks yang sesuai
result = pd.DataFrame(index=df3.index, columns=unique_values)

# Mengisi nilai default dengan 'N'
result = result.fillna('N')

# Menetapkan nilai 'Y' sesuai keberadaan nilai dalam DataFrame asli untuk kolom yang diinginkan
for col in unique_values:
    for loc_col in df3.columns:
        result[col] = result[col].mask(df3[loc_col] == col, 'Y')
df3 = pd.concat([df3, result], axis=1)

In [76]:
df3.head()

,Keywords,Address,harga,kamar_tidur,bangunan,lahan,Title,Field,Text1,Text2,...,Universitas Atmajaya Yogyakarta,rs sardjito yogyakarta,SMAN 6 Yogyakarta,STMIK AMIKOM YOGYAKARTA,Universitas Islam Negeri Sunan Kalijaga,Gereja Saint Francis Xaverius Yogyakarta,Sahid J-Walk,Rumah Sakit Panti Rapih,Universitas Sanata Dharma Yogyakarta,SMAN 3 Yogyakarta
0,Premium ...,Yogyakarta,825000000,2,40,66,Yogyakarta,Rumah Baru Siap Huni Premium Promo Ramadhan,rincian ...,Fasilitas ...,...,N,N,N,N,N,N,N,N,N,N
1,Premium ...,"Jetis, Yogyakarta",1680000000,4,140,500,"Jetis, Yogyakarta",Dijual Cepat Rumah Bule di Yogyakarta,rincian ...,Fasilitas ...,...,N,N,N,N,N,N,N,N,N,N
2,Premium ...,Yogyakarta,930000000,2,66,92,"Jalan Gedongan Kuning Selatan, Purbayan Kel., ...",Rumah Baru Siap Huni Premiun Ramadhan,rincian ...,Fasilitas ...,...,N,N,N,N,N,N,N,N,N,N
3,Premium ...,"Mergangsan, Yogyakarta",4500000000,4,350,230,jalan parangtritis jogja ...,Rumah Mewah furnish dalam perumahan elit Lagun...,rincian ...,Fasilitas ...,...,N,N,N,N,N,N,N,N,N,N
4,New ...,"Kraton, Yogyakarta",1300000000,3,90,100,"Kraton, Yogyakarta",TERMURAH! Rumah Dalam Beteng Kraton Yogya Hany...,rincian ...,NaN,...,N,N,N,N,N,Y,N,N,N,N


In [80]:
df4 = df3.copy()

In [81]:
df4['jenis_rumah'] = df4['Field'].apply(lambda x: 'kost' if 'kost' in str(x).lower() or 'kosan' in str(x).lower() else
                                          'cluster' if 'cluster' in str(x).lower() else
                                          'townhouse' if 'townhouse' in str(x).lower() or 'town house' in str(x).lower() else
                                          'penginapan' if 'homestay' in str(x).lower() or 'penginapan' in str(x).lower() or ' home stay' in str(x).lower() or 'villa' in str(x).lower() else
                                           'ruko' if 'ruko' in str(x).lower() or 'toko' in str(x).lower() or 'rumah toko' in str(x).lower() else
                                            'kantor' if 'kantor' in str(x).lower() or 'rumah kantor' in str(x).lower() else
                                           'residence' if 'residence' in str(x).lower() or 'rumah residence' in str(x).lower() or 'elit' in str(x).lower() or 'elite' in str(x).lower() else
                                           'subsidi' if 'subsidi' in str(x).lower() or 'rumah subsidi' in str(x).lower() else
                                           'perumahan' if 'perumahan' in str(x).lower() else 'rumah biasa')

In [82]:
df4['kategori'] = df4[['Field', 'Keywords']].apply(lambda row: 'premium' if 'premium' in str(row['Field']).lower() or 'premium' in str(row['Keywords']).lower() else
                                                     'premium' if 'mewah' in str(row['Field']) else
                                                     'premium' if 'eksklusif' in str(row['Field']) else
                                                     'premium' if 'elit' in str(row['Field']) else
                                                    'premium' if 'elite' in str(row['Field']) else
                                                     'biasa', axis=1)

In [83]:
df4 = df4.drop(['Text1','Text2','Text3','Keywords','Field','lokasi 1','lokasi 2','lokasi 3','lokasi 4','lokasi 5'],axis=1)

In [84]:
df4 = df4.rename(columns={"Address": "lokasi",'Title':'alamat'})

In [85]:
df4['Lahan Parkir'] = df4['Lahan Parkir'].replace({None: 0})

In [86]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1495 entries, 0 to 1499
Data columns (total 78 columns):
 #   Column                                                         Non-Null Count  Dtype 
---  ------                                                         --------------  ----- 
 0   lokasi                                                         1495 non-null   object
 1   harga                                                          1495 non-null   int64 
 2   kamar_tidur                                                    1495 non-null   int64 
 3   bangunan                                                       1495 non-null   object
 4   lahan                                                          1492 non-null   object
 5   alamat                                                         1495 non-null   object
 6   Lahan Parkir                                                   1495 non-null   object
 7   Jumlah Lantai                                                  369 no

In [87]:
df4.drop(columns=['Jumlah Lantai','Lahan Parkir'], inplace=True)

In [88]:
df4.head()

,lokasi,harga,kamar_tidur,bangunan,lahan,alamat,Kamar mandi,Sistem Alarm,Gym,Internet broadband/Wifi,...,SMAN 6 Yogyakarta,STMIK AMIKOM YOGYAKARTA,Universitas Islam Negeri Sunan Kalijaga,Gereja Saint Francis Xaverius Yogyakarta,Sahid J-Walk,Rumah Sakit Panti Rapih,Universitas Sanata Dharma Yogyakarta,SMAN 3 Yogyakarta,jenis_rumah,kategori
0,Yogyakarta,825000000,2,40,66,Yogyakarta,2,N,N,N,...,N,N,N,N,N,N,N,N,rumah biasa,premium
1,"Jetis, Yogyakarta",1680000000,4,140,500,"Jetis, Yogyakarta",5,N,N,N,...,N,N,N,N,N,N,N,N,rumah biasa,premium
2,Yogyakarta,930000000,2,66,92,"Jalan Gedongan Kuning Selatan, Purbayan Kel., ...",2,N,N,N,...,N,N,N,N,N,N,N,N,rumah biasa,premium
3,"Mergangsan, Yogyakarta",4500000000,4,350,230,jalan parangtritis jogja ...,4,Y,Y,Y,...,N,N,N,N,N,N,N,N,residence,premium
4,"Kraton, Yogyakarta",1300000000,3,90,100,"Kraton, Yogyakarta",1,N,N,N,...,N,N,N,Y,N,N,N,N,rumah biasa,biasa


In [89]:
df4.to_csv('jog.csv')